In [91]:
import bonesis
import bonesis.aeon
import biodivine_aeon
import csv
import textwrap

In [92]:
aeon = "toy.aeon"
data_loc = "toy.tsv"

In [93]:
with open(aeon, "w") as fp:
    fp.write(textwrap.dedent("""\
        #name:toy
        #rule for A is fixed
        $A:!A | C & T
        A -| A
        C -> A
        T -> A
        #C regulates B
        A -|? B
        B -|? B
        C -> B
        #(A, B) is grouped for C
        $AB: A & B
        A -> AB
        B -> AB
        AB -?? C
        #T is the source
        $T:T
        T -> T"""))

with open(data_loc, "w") as fp:
    fp.write(textwrap.dedent("""\
        ID\tScore\tSource\tPerturbation\tObserved node\tCategorization
        1\t1.0\tT=1\t\tA\tON
        2\t1.0\tT=1\t\tB\tON
        3\t1.0\tT=1\t\tC\tON
        4\t1.0\tT=1\tC KO\tB\tOFF"""))

In [94]:
aeon_bn = biodivine_aeon.BooleanNetwork.from_file(aeon)

aeon_bn.explicit_parameter_count()

rg = aeon_bn.to_graph()


print(len(rg.variable_names()))

data = {}
for node in rg.variable_names():
    data[node + "_OFF"] = {node:0}
    data[node + "_ON"] = {node:1}


print(data)

5
{'A_OFF': {'A': 0}, 'A_ON': {'A': 1}, 'AB_OFF': {'AB': 0}, 'AB_ON': {'AB': 1}, 'B_OFF': {'B': 0}, 'B_ON': {'B': 1}, 'C_OFF': {'C': 0}, 'C_ON': {'C': 1}, 'T_OFF': {'T': 0}, 'T_ON': {'T': 1}}


In [95]:
dom = bonesis.aeon.AEONDomain.from_aeon_file(aeon)

bo = bonesis.BoNesis(dom, data)

In [96]:
bo.is_satisfiable()

True

In [97]:
model_name = "toy"

print(bo.boolean_networks().count())

i=0
for bn in bo.boolean_networks(limit=10):
    i+=1
    # if not os.path.exists(f"{model_name}"):
    #     os.makedirs(f"{model_name}")
    # bn.save(f"{model_name}/bn-solution_{i}.bnet")

    print("-"*20)
    print(bn)
    for att in list(bn.attractors()):
        print(att)

Grounding...done in 0.0s
56
Grounding...done in 0.0s
--------------------
A <- !A|(C&T)
AB <- A&B
B <- C
C <- 1
T <- T

{'A': 1, 'AB': 1, 'B': 1, 'C': 1, 'T': 1}
{'A': '*', 'AB': '*', 'B': 1, 'C': 1, 'T': 0}
--------------------
A <- !A|(C&T)
AB <- A&B
B <- C
C <- !AB
T <- T

{'A': '*', 'AB': '*', 'B': '*', 'C': '*', 'T': 0}
{'A': '*', 'AB': '*', 'B': '*', 'C': '*', 'T': 1}
--------------------
A <- !A|(C&T)
AB <- A&B
B <- C
C <- AB
T <- T

{'A': 1, 'AB': 1, 'B': 1, 'C': 1, 'T': 1}
{'A': '*', 'AB': 0, 'B': 0, 'C': 0, 'T': 0}
{'A': '*', 'AB': 0, 'B': 0, 'C': 0, 'T': 1}
--------------------
A <- !A|(C&T)
AB <- A&B
B <- C
C <- 0
T <- T

{'A': '*', 'AB': 0, 'B': 0, 'C': 0, 'T': 0}
{'A': '*', 'AB': 0, 'B': 0, 'C': 0, 'T': 1}
--------------------
A <- !A|(C&T)
AB <- A&B
B <- !B&C
C <- 1
T <- T

{'A': 1, 'AB': '*', 'B': '*', 'C': 1, 'T': 1}
{'A': '*', 'AB': '*', 'B': '*', 'C': 1, 'T': 0}
--------------------
A <- !A|(C&T)
AB <- A&B
B <- !B&C
C <- AB
T <- T

{'A': '*', 'AB': 0, 'B': 0, 'C': 0,

In [98]:
def comment_removal(line:str) -> bool:
    if line.startswith("#"):
        print(line)
    return not line.startswith("#") and not line.isspace()

def get_bo(aeon, data, data_loc, skip_list):

    dom = bonesis.aeon.AEONDomain.from_aeon_file(aeon)

    bo = bonesis.BoNesis(dom, data)


    ID, SCORE, SOURCE, PERT, NODE, VALUE = 0, 1, 2, 3, 4, 5

    file = open(data_loc, "r")
    lines = filter(comment_removal, file)
    data_csv = csv.reader(lines, delimiter="\t")

    # skip the first row
    next(data_csv)

    i = 0
    for row in data_csv:

        # ignore any data with "Some"
        if row[VALUE] in ["Some", "Some/ON", "OFF/Some"]:
            continue

        if int(row[ID]) in skip_list:
            continue
        
        i+=1

        if row[VALUE] == "ON":
            node_state = row[NODE] + "_ON"
        elif row[VALUE] == "OFF":
            node_state = row[NODE] + "_OFF"
        else:
            print("Unexpected value", row[VALUE])
            raise Exception("Unexpected experiment output")

        # wild type
        if row[SOURCE] == "" and row[PERT] == "":
            bo.fixed(bo.obs(node_state))
            bo.all_fixpoints(bo.obs(node_state))

        # perturbations
        else:
            fixes_dict = {}
            if row[SOURCE] != "":
                # add source node values to the fixes
                source_str = row[SOURCE].split(",")
                for sth in source_str:
                    node, value = sth.strip().split("=")
                    fixes_dict[node] = int(value)
        
            if row[PERT] != "":
                # add other perturbations to the fixes
                pert_str = row[PERT].split(",")
                for sth in pert_str:
                    node, value_str = sth.strip().split(" ")
                    if value_str == "KO":
                        value = 0
                    elif value_str == "CA":
                        value = 1
                    else:
                        raise Exception("Perturbation should be KO or CA")
                    fixes_dict[node] = int(value)

            with bo.mutant(fixes_dict):
                bo.fixed(bo.obs(node_state))
                bo.all_fixpoints(bo.obs(node_state))

    print("incorporated", i, "experiments")
    
    file.close()


    return bo

In [99]:
bo = get_bo(aeon, data, data_loc, [])

bo.is_satisfiable()

incorporated 4 experiments


True

In [100]:
model_name = "toy"

print(bo.boolean_networks().count())

i=0
for bn in bo.boolean_networks(limit=11):
    i+=1
    # if not os.path.exists(f"{model_name}"):
    #     os.makedirs(f"{model_name}")
    # bn.save(f"{model_name}/bn-solution_{i}.bnet")

    print("-"*20)
    print(bn)
    for att in list(bn.attractors()):
        print(att)

Grounding...done in 0.0s
2
Grounding...done in 0.0s
--------------------
A <- !A|(C&T)
AB <- A&B
B <- C
C <- 1
T <- T

{'A': 1, 'AB': 1, 'B': 1, 'C': 1, 'T': 1}
{'A': '*', 'AB': '*', 'B': 1, 'C': 1, 'T': 0}
--------------------
A <- !A|(C&T)
AB <- A&B
B <- C
C <- AB
T <- T

{'A': 1, 'AB': 1, 'B': 1, 'C': 1, 'T': 1}
{'A': '*', 'AB': 0, 'B': 0, 'C': 0, 'T': 0}
{'A': '*', 'AB': 0, 'B': 0, 'C': 0, 'T': 1}
